<a href="https://colab.research.google.com/github/gab-palmeri/aml-geolocalization/blob/sam/step_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to test the ensemble of two or more networks where the logits are the mean of the logits of each network

# pip install requirements

Remember to click on "Restart Runtime" before go on

In [ ]:
# CosPlace requirements
!pip3 install "faiss_cpu>=1.7.1"
!pip3 install "numpy>=1.21.2"
!pip3 install "Pillow>=9.0.1"
!pip3 install "scikit_learn>=1.0.2"
!pip3 install "torch>=1.8.2"
!pip3 install "torchvision>=0.9.2"
!pip3 install "tqdm>=4.62.3"
!pip3 install "utm>=0.7.0"

import torch
#use GPU if available 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

# Download Datasets and previous data

Downloading with gdown doesn't work properly because after N downloads it stops working. So we prefer to use drive / mount with a shared folder.

In [ ]:
use_mount = True

In [ ]:
import os
import gdown
from google.colab import drive

def download(id, output=None, quiet=True):
  gdown.download(
    f"https://drive.google.com/uc?export=download&confirm=pbef&id={id}",
    output=output,
    quiet=quiet
  )


if use_mount:
  drive.mount('/content/drive')

# TOKYO-XS DATASET
if not os.path.isdir("/content/tokyo_xs"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-xs.zip"
  else:
    id = "1fBCnap5BRh36474cVkjvjlC-yUTEb1n3"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-xs.zip"                    # unzip
    !rm -r "/content/tokyo-xs.zip"                      # remove .zip file

if not os.path.isdir("/content/tokyo_xs"):
  raise FileNotFoundError(f"Can't download tokyo xs")

#TOKYO NIGHT DATASET
if not os.path.isdir("/content/tokyo-night"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-night.zip"
  else:
    id = "1EZJY2r5565-iVk2oEbTns0xc4F_em4Y4"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-night.zip"                    # unzip
    !rm -r "/content/tokyo-night.zip"

if not os.path.isdir("/content/tokyo-night"):
  raise FileNotFoundError(f"Can't download tokyo night")

# SAN FRANCISCO - XS DATASET
if not os.path.isdir("/content/small"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/sf-xs.zip"
  else:
    id = "1brIxBJmOgvuzFbI57f5LxnMxjccUu993"
    download(id, quiet=False)                           # download
    !jar xvf "/content/sf-xs.zip"                       # unzip
    !rm -r "/content/sf-xs.zip"                         # remove .zip file

if not os.path.isdir("/content/small"):
  raise FileNotFoundError(f"Can't download sfxs")

# MODEL TO ENSEMBLE
if not os.path.isdir("/content/content"):
  if use_mount:
    !jar xvf "/content/myDrive/MyDrive/shared/models_to_ensemble.zip"
  else:
    id = "1hDBK9y4bzvT90cnRdJ5OzkgkkmQAwtmE"
    download(id, quiet=False)                           # download
    !jar xvf "/content/models_to_ensemble.zip"          # unzip
    !rm -r "/content/models_to_ensemble.zip"            # remove .zip file

# Download Code

Clone of original repo of CosPlace and our code

In [ ]:
# download code of CosPlace
!git clone "https://github.com/gmberton/CosPlace" 
#!rm -r "/content/CosPlace"

# download our code
!git clone --single-branch --branch "develop" "https://github.com/gab-palmeri/aml-geolocalization.git"
!mv "/content/aml-geolocalization/" "/content/Team"
#!rm -r "/content/aml-geolocalization"



# Import Code


In [ ]:
import os
import sys
import torch
import logging
import multiprocessing
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm
from datetime import datetime

sys.path.append("/content/CosPlace/")
sys.path.append("/content/Team/")
import CosPlace
from CosPlace import *

torch.backends.cudnn.benchmark = True  # Provides a speedup

This class let us to access to dictionary keys like `dict.key` instead of `dict["key"]`

In [ ]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

Drive related functions to save and load training checkpoints

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

def drive_save_checkpoint(output_folder, time=None, drive_folder="/content/drive/MyDrive/project6/"):
    if time is None:
        raise ValueError("time must be specified")
    if not os.path.exists(drive_folder):
        !mkdir "{drive_folder}"
    !zip -r "{drive_folder}{time}.zip" "/content/{output_folder}"

def drive_load_checkpoint(drive_folder="/content/drive/MyDrive/project6/", time=None):
    if time is None:
        raise ValueError("time must be specified")
    !jar xvf "{input_folder}{time}.zip"

def drive_tester(output_folder, drive_folder="/content/drive/MyDrive/project6/"):
    if not os.path.exists(drive_folder):
        !mkdir "{drive_folder}"
    !touch "{drive_folder}test"

    if os.path.exists("{drive_folder}test"):
        logging.info("Drive saving works")
    else:
        logging.info("WARNING: Drive saving does not work")

# use this function when content on drive are not updated
def drive_refresh():
  drive.flush_and_unmount()
  drive.mount('/content/drive')

# Parameters

Original code provides two main executable files: `train.py` and `eval.py`. We want to reproduce the same behaviour of these two files so these are the parameters passed via terminal. They will be put in a dict called `args` to simply reuse code inside ex files

## Setup parameters

In [ ]:
# CosPlace Groups parameters
COS_M = 10
ALPHA = 30
COS_N = 5
COS_L = 2
GROUPS_NUM = 1
MIN_IMAGES_PER_CLASS = 10
# Model parameters
BACKBONE = "resnet18"   
# ["resnet18", "efficientnet_b2", "efficientnet_v2_s", "mobilenet_v3_small", "mobilenet_v3_large",
# ["convnext_tiny", "swin_tiny" ]
FC_OUTPUT_DIM = 512     # Output dimension of final fully connected layer
PRETRAIN = "imagenet"   # ["imagenet", "places", "gldv2"]
# Training parameters
AUGMENTATION_DEVICE = "cuda"    # ["cuda", "cpu"]
USE_AMP_16 = AUGMENTATION_DEVICE == "cuda"       # use Automatic Mixed Precision
BATCH_SIZE = 32
EPOCHS_NUM = 3
ITERATIONS_PER_EPOCH = 10_000
LR = 0.00001                    # Learning rate  
CLASSIFIERS_LR = 0.01
# Data augmentation
RESIZE_AS_DB = True
BRIGHTNESS = 0.7
CONTRAST = 0.7
SATURATION = 0.7
HUE = 0.5
RANDOM_RESIZED_CROP = 0.5
RANDOM_H_FLIP = False
MAJORITY_WEIGHT = 0.01
# Validation / test parameters
INFER_BATCH_SIZE = 16           # Batch size for inference (validating and testing)
POSITIVE_DIST_THRESHOLD = 25    # distance in meters for a prediction to be considered a positive
# Resume parameters
RESUME_TRAIN = None     # path to checkpoint to resume, e.g. logs/.../last_checkpoint.pth
RESUME_MODEL = None     # Path to model to resume training from
# Other parameters
DEVICE = "cuda"                     # ["cuda", "cpu"]
SEED = 0
NUM_WORKERS = 8
DATASET_FOLDER = "/content/small"   # path of the folder with train/val sets
SAVEDIR = BACKBONE + "_" + PRETRAIN
TEST_METHOD = "central_crop"


if not os.path.exists(DATASET_FOLDER):
    raise FileNotFoundError(f"Dataset folder {DATASET_FOLDER} not found")

train_set_folder = os.path.join(DATASET_FOLDER, "train")

if not os.path.exists(train_set_folder):
    raise FileNotFoundError(f"Train set folder {train_set_folder} not found")

val_set_folder = os.path.join(DATASET_FOLDER, "val")

if not os.path.exists(val_set_folder):
    raise FileNotFoundError(f"Validation set folder {val_set_folder} not found")

if BACKBONE != "resnet18" and PRETRAIN != "imagenet":
    raise ValueError("Only resnet18 can be pretrained on other datasets than ImageNet")

# dictionary for the parameters
args = {
    'M': COS_M, 'alpha': ALPHA, 'N': COS_N, 'L': COS_L, 'groups_num': GROUPS_NUM,
    'min_images_per_class': MIN_IMAGES_PER_CLASS, 'backbone': BACKBONE, "pretrain": PRETRAIN,
    'fc_output_dim': FC_OUTPUT_DIM, 'use_amp16': USE_AMP_16,
    'augmentation_device': AUGMENTATION_DEVICE, 'batch_size': BATCH_SIZE,
    'epochs_num': EPOCHS_NUM, 'iterations_per_epoch': ITERATIONS_PER_EPOCH,
    'lr': LR, 'classifiers_lr': CLASSIFIERS_LR, 'brightness': BRIGHTNESS, 'resize_as_db': RESIZE_AS_DB,
    'contrast': CONTRAST, 'hue': HUE, 'saturation': SATURATION, 'hflip': RANDOM_H_FLIP, 'maj_weight': MAJORITY_WEIGHT,
    'random_resized_crop': RANDOM_RESIZED_CROP, 'infer_batch_size': INFER_BATCH_SIZE,
    'positive_dist_threshold': POSITIVE_DIST_THRESHOLD, 'resume_train': RESUME_TRAIN,
    'resume_model': RESUME_MODEL, 'device': DEVICE, 'seed': SEED,
    'num_workers': NUM_WORKERS, 'dataset_folder': DATASET_FOLDER, 'save_dir': SAVEDIR,
    'val_set_folder': val_set_folder, 'train_set_folder': train_set_folder, 'test_method': TEST_METHOD
}

# this helps to reuse the code from the original CosPlace
args = dotdict(args)


In [ ]:
saveDirLocal = SAVEDIR

# Setup logging

In [ ]:
from CosPlace import commons

start_time = datetime.now()
output_folder = f"logs/{args.save_dir}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}"
commons.make_deterministic(args.seed)
commons.setup_logging(output_folder, console=None)
logging.info(" ".join(sys.argv))
logging.info(f"Arguments: {args}")
logging.info(f"The outputs are being saved in {output_folder}")

# Test

## Models to ensemble

In [ ]:
import os
from Team.model.network import GeoLocalizationNet

# define paths of state dicts
paths = [
    "/content/content/resnet18_imagenet.pth",
    "/content/content/mobilenet_v3_large_imagenet.pth",
    "/content/content/efficientnet_v2_s_imagenet.pth"
]

for path in paths:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Model {path} not found, you have to provide the state dicts of the models")

state_dicts = [torch.load(x) for x in paths]


# create models

models = [
    GeoLocalizationNet("resnet18", args.fc_output_dim),
    GeoLocalizationNet("mobilenet_v3_large", args.fc_output_dim),
    GeoLocalizationNet("efficientnet_v2_s", args.fc_output_dim)
]

# load state dicts
for i, model in enumerate(models):
    model.load_state_dict(state_dicts[i])
    model = model.to(args.device)

## Import model

In [ ]:
from Team.model.ensemble import ModelEnsembler


model = ModelEnsembler(models)
model = model.to(args.device)

## Test on SF-XS

Test the model on the sf-xs (test) dataset.

In [ ]:
from Team.datasets.test_dataset import DataAugTestDataset as TestDataset
from Team import test

# dataset_folder is the same of the training
test_set_folder = os.path.join(DATASET_FOLDER, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder, queries_folder="queries_v1",
                      positive_dist_threshold=args.positive_dist_threshold, test_method=args.test_method, resize = args.resize_as_db)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
sf_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

## Test on Tokyo-XS

Test the model on the tokyo-xs dataset.

In [ ]:
tokyo_xs_folder = "/content/tokyo_xs"
test_set_folder = os.path.join(tokyo_xs_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold, test_method=args.test_method, resize = args.resize_as_db)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

## Test on Tokyo-Night

Test the model on the tokyo-night dataset.

In [ ]:
tokyo_night_folder = "/content/tokyo-night/"
test_set_folder = os.path.join(tokyo_night_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold, test_method=args.test_method, resize = args.resize_as_db)

recalls, recalls_str = test.test(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_night_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

# Save results

## Create CSV with recalls

In [ ]:
import csv
header = ["sf-xs (test)", "Tokyo-xs", "Tokyo-night"]
data = [sf_xs_r15, tokyo_xs_r15, tokyo_night_r15]

for h, d in zip(header, data):
  logging.info(f"{h}: {d}")

with open(f"/content/{SAVEDIR}_{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv", "w") as f:
  writer = csv.writer(f)

  writer.writerow(header)
  writer.writerow(data)
logging.info(f"save table results to /content/{SAVEDIR}{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.csv")